In [2]:
!pip3 install --upgrade openai

In [3]:
import pandas as pd
data = pd.read_json('cladder.json')
data = data.dropna().reset_index(drop=True)

data = data.assign(step0 = lambda x: (x['reasoning']))
df = data['step0'].apply(lambda x: (x.get("step0")))
data['step0'] = df

data = data.assign(step1 = lambda x: (x['reasoning']))
df = data['step1'].apply(lambda x: (x.get("step1")))
data['step1'] = df

#step2) query type
data = data.assign(step2 = lambda x: (x['meta']))
df = data['step2'].apply(lambda x: (x.get("query_type")))
data['step2'] = df

#step3) formalize query
data = data.assign(step3 = lambda x: (x['reasoning']))
df = data['step3'].apply(lambda x: (x.get("step2")))
data['step3'] = df

#step4) extract all available data
data = data.assign(step4 = lambda x: (x['reasoning']))
df = data['step4'].apply(lambda x: (x.get("step4")))
data['step4'] = df

#step5) deduce estimand
data = data.assign(step5 = lambda x: (x['reasoning']))
df = data['step5'].apply(lambda x: (x.get("step3")))
data['step5'] = df

#step6)Insert the relevant data in Step 4 into the estimand, perform basic arithmetic calculations
data = data.assign(step6 = lambda x: (x['reasoning']))
df = data['step6'].apply(lambda x: (x.get("step5")))
data['step6'] = df

#end) derive the final answer
data = data.assign(end = lambda x: (x['reasoning']))
df = data['end'].apply(lambda x: (x.get("end")))
data['end'] = df

data

,question_id,desc_id,given_info,question,answer,meta,reasoning,step0,step1,step2,step3,step4,step5,step6,end
0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",nde,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.08\nP(Y=1 | X=0, V2=1) ...","\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y...",0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = ...,0.32 > 0
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",ate,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.26\nP(Y=1 | X=1) = 0.76,P(Y=1|X=1) - P(Y=1|X=0),0.76 - 0.26 = 0.50,0.50 > 0
2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",marginal,P(Y),P(X=1) = 0.77\nP(Y=1 | X=0) = 0.26\nP(Y=1 | X=...,P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0),0.77*0.76 - 0.23*0.26 = 0.64,0.64 > 0
3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",ate,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.20\nP(Y=1 | X=1) = 0.68,P(Y=1|X=1) - P(Y=1|X=0),0.68 - 0.20 = 0.49,0.49 > 0
4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",nie,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.11\nP(Y=1 | X=0, V2=1) ...","\sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X...",0.01 * (0.60 - 0.11)+ 0.61 * (0.92 - 0.46)= -0.29,-0.29 < 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",det-counterfactual,Y_{X=0} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 0 or 1,1
8528,31014,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if zuph instead o...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",det-counterfactual,Y_{X=1} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 1 or 1,1
8529,31015,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is not glimx if zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",det-counterfactual,Y_{X=1} = 0 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 1 or 1,0
8530,31016,nonsense9-fork-det-counterfactual-model3843-sp...,We know that zuph and jyka causes glimx. We ob...,Would an individual is glimx

In [4]:
#create a list of real variable names and their respective representation e.g. [['X','wife'],['Y','husband']]
def list_representation(x):
    x = x.get('step0')
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    aux = []
    for i in range(len(x)):
        aux.append(x[i].split(" = "))
    return aux

#receives a list from list_representation and returns the prompt with it
def prompt_repres(x):
    x = x.get('step0')
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    list_rep = []
    for i in range(len(x)):
        list_rep.append(x[i].split(" = "))
    
    prompt = ""
    for i in range(len(list_rep)):
        prompt += "Use "+list_rep[i][0]+" to represent "+list_rep[i][1]+". "
    return prompt

def create_string(str_list):
    string = ""
    for i in range(str_list):
        string += str_list[i]
    return string

#exchange query_type abbreviation to the whole word
def expand(abb):
    if abb == 'nde':
        return 'natural direct effect'
    elif abb == 'ate':
        return 'average treatment effect'
    elif abb == 'marginal':
        return 'marginal probability'
    elif abb == 'nie':
        return 'natural indirect effect'
    elif abb == 'det-counterfactual':
        return 'normal counterfactual question'
    elif abb == 'ett':
        return 'average treatment effect on treated'
    elif abb == 'correlation':
        return 'conditional probability'
    elif abb == 'exp_away':
        return 'explaining away effect'
    elif abb == 'collider_bias':
        return 'collider bias'
    elif abb == 'backadj':
        return 'backdoor adjustment set'
    return 0
    

#data['formal_form'] = data.apply(lambda x: exchange(x['reasoning'], x['meta'],1), axis=1)
#data['graph'] = data.apply(lambda x: exchange(x['reasoning'], x['reasoning'],0), axis=1)
#data['info'] = data.apply(lambda x: exchange(x['reasoning'], x['info'],2), axis=1)
data['step2'] = data.apply(lambda x: expand(x['step2']), axis=1)

#data

In [5]:
#create columnn with complete prompt to be given to the LLM
paraphrases = [
            "Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships. ",
            "Think of a self-contained, hypothetical setting with just the specified conditions, and devoid of any unknown factors or causal connections. ",
            "Consider a self-contained, hypothetical world with solely the mentioned conditions, and is free from any hidden factors or cause-and-effect relationships. ",
            "Imagine a self-contained, hypothetical setting with merely the stated conditions, and absent any unmentioned factors or causative links. ",
            "Think of a self-contained, hypothetical world with only the given conditions, and is void of any unknown factors or causative relationships. ",
        ]

prompt_end0 = " Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that\
depicts the relationships in the scenario. "
prompt_end1 = "The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query \
type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining\
away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment\
effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed \
in quotation marks. Step 3) Extract all the available data. Your answer should contain nothing but marginal probabilities and conditional \
probabilities in the form 'P(...)=...' or 'P(...|...)=...', each probability being separated by a semicolon. Stick to the previously \
mentioned denotations for the variables. Based on all the reasoning above, output <YES> or <NO> to answer the following question: "

'''"The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine \
the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', \
'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', \
'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the \
list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on \
its type, utilizing the 'do(·)' notation or counterfactual notations as needed. Step 4) Extract all the available data. Your answer should \
contain nothing but marginal probabilities and conditional probabilities in the form 'P(...)=...' or 'P(...|...)=...', each probability\
being separated by a semicolon. Stick to the previously mentioned denotations for the variables. Step 5) Given all the information above, \
deduce the estimand using skills such as do-calculus, counterfactual prediction, and the basics of probabilities. Answer step by step. \
Step 6) Insert the relevant data in Step 4 into the estimand, perform basic arithmetic calculations, and derive the final answer. There \
is an identifiable answer. Answer step by step. \nBased on all the reasoning above, output one word to answer the initial question."'''


data['prompt'] = data.apply(lambda x: (paraphrases[0] + 'Question: ' + x['given_info']+prompt_end0+prompt_repres(x['reasoning'])+prompt_end1+x['question']),axis=1)

#df = data[['prompt', 'formal_form', 'graph', 'query_type']]
data

,question_id,desc_id,given_info,question,answer,meta,reasoning,step0,step1,step2,step3,step4,step5,step6,end,prompt
0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",natural direct effect,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.08\nP(Y=1 | X=0, V2=1) ...","\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y...",0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = ...,0.32 > 0,"Imagine a self-contained, hypothetical world w..."
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",average treatment effect,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.26\nP(Y=1 | X=1) = 0.76,P(Y=1|X=1) - P(Y=1|X=0),0.76 - 0.26 = 0.50,0.50 > 0,"Imagine a self-contained, hypothetical world w..."
2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",marginal probability,P(Y),P(X=1) = 0.77\nP(Y=1 | X=0) = 0.26\nP(Y=1 | X=...,P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0),0.77*0.76 - 0.23*0.26 = 0.64,0.64 > 0,"Imagine a self-contained, hypothetical world w..."
3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",average treatment effect,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.20\nP(Y=1 | X=1) = 0.68,P(Y=1|X=1) - P(Y=1|X=0),0.68 - 0.20 = 0.49,0.49 > 0,"Imagine a self-contained, hypothetical world w..."
4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",natural indirect effect,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.11\nP(Y=1 | X=0, V2=1) ...","\sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X...",0.01 * (0.60 - 0.11)+ 0.61 * (0.92 - 0.46)= -0.29,-0.29 < 0,"Imagine a self-contained, hypothetical world w..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",normal counterfactual question,Y_{X=0} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 0 or 1,1,"Imagine a self-contained, hypothetical world w..."
8528,31014,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if zuph instead o...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",normal counterfactual question,Y_{X=1} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 1 or 1,1,"Imagine a self-contained, hypothetical world w..."
8529,31015,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka 

In [6]:
from openai import OpenAI

client = OpenAI()
# defaults to getting the key using os.environ.get("OPENAI_API_KEY")
# if you saved the key under a different environment variable name, you can do something like:
# client = OpenAI(
#   api_key=os.environ.get("CUSTOM_ENV_NAME"),
# )

In [12]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are an expert in causal inference. The following question is not a typical commonsense query, but rather a meticulously designed question created by a professor specializing in causal inference, intended to assess the students' mastery of the course content."},
    {"role": "user", "content": "Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships. Question: For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 9%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%. Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph thatdepicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explainingaway effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatmenteffect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Extract all the available data. Your answer should contain nothing but marginal probabilities and conditional probabilities in the form 'P(...)=...' or 'P(...|...)=...', each probability being separated by a semicolon. Stick to the previously mentioned denotations for the variables. Based on all the reasoning above, output <YES> or <NO> to answer the following question: If we disregard the mediation effect through wife, would husband positively affect alarm clock?"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Step 1) Causal graph: X->Y, V2->Y, X->V2\nStep 2) Query type: "natural direct effect"\nStep 3) Available data:\nP(Y|X=0,V2=0)=0.09; P(Y|X=0,V2=1)=0.54; P(Y|X=1,V2=0)=0.41; P(Y|X=1,V2=1)=0.86;\nP(V2=1|X=0)=0.74; P(V2=1|X=1)=0.24;\n\nThe natural direct effect of X on Y is the effect of changing X from 0 to 1 with the wife\'s alarm-setting behavior held constant. In this scenario, the natural direct effect is calculated as P(Y|do(X=1),V2) - P(Y|do(X=0),V2). \n\nFrom the available data, we can see that if we disregard the mediation effect through the wife (holding her behavior constant), the impact of husbands setting the alarm positively affects the alarm clock. Therefore, the answer is <YES>.', role='assistant', function_call=None, tool_calls=None)


In [22]:
text=str(completion.choices[0].message)
text[31:]

'Step 1) Causal graph: X->Y, V2->Y, X->V2\\nStep 2) Query type: "natural direct effect"\\nStep 3) Available data:\\nP(Y|X=0,V2=0)=0.09; P(Y|X=0,V2=1)=0.54; P(Y|X=1,V2=0)=0.41; P(Y|X=1,V2=1)=0.86;\\nP(V2=1|X=0)=0.74; P(V2=1|X=1)=0.24;\\n\\nThe natural direct effect of X on Y is the effect of changing X from 0 to 1 with the wife\\\'s alarm-setting behavior held constant. In this scenario, the natural direct effect is calculated as P(Y|do(X=1),V2) - P(Y|do(X=0),V2). \\n\\nFrom the available data, we can see that if we disregard the mediation effect through the wife (holding her behavior constant), the impact of husbands setting the alarm positively affects the alarm clock. Therefore, the answer is <YES>.\', role=\'assistant\', function_call=None, tool_calls=None)'

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
data["prompt"].iloc[0]

"Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships. Question: For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%. Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph thatdepicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm cl

In [2]:
import pandas as pd

In [3]:
data=pd.read_csv('/media/data/bazaluk/cladder_gpt3.5.csv')

In [3]:
data

,Unnamed: 0,question_id,desc_id,given_info,question,answer,meta,reasoning,step0,step1,...,step3,step4,step5,step6,end,prompt,predicted graph,predicted query type,predicted final answer,predicted text
0,0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.08\nP(Y=1 | X=0, V2=1) ...","\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y...",0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = ...,0.32 > 0,"Imagine a self-contained, hypothetical world w...",") Causal graph: X -> Y, V2 -> Y, X -> V2\n\n",average treatment effect,yes,ChatCompletionMessage(content='Step 1) Causal ...
1,1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.26\nP(Y=1 | X=1) = 0.76,P(Y=1|X=1) - P(Y=1|X=0),0.76 - 0.26 = 0.50,0.50 > 0,"Imagine a self-contained, hypothetical world w...",) Causal graph: X -> Y\n\n,conditional probability,yes,ChatCompletionMessage(content='Step 1) Causal ...
2,2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,P(Y),P(X=1) = 0.77\nP(Y=1 | X=0) = 0.26\nP(Y=1 | X=...,P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0),0.77*0.76 - 0.23*0.26 = 0.64,0.64 > 0,"Imagine a self-contained, hypothetical world w...",") Causal graph: X -> Y, V2 -> Y \n",marginal probability,yes,ChatCompletionMessage(content='Step 1) Causal ...
3,3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.20\nP(Y=1 | X=1) = 0.68,P(Y=1|X=1) - P(Y=1|X=0),0.68 - 0.20 = 0.49,0.49 > 0,"Imagine a self-contained, hypothetical world w...",) Causal graph: X -> Y \n,conditional probability,yes,ChatCompletionMessage(content='Step 1) Causal ...
4,4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.11\nP(Y=1 | X=0, V2=1) ...","\sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X...",0.01 * (0.60 - 0.11)+ 0.61 * (0.92 - 0.46)= -0.29,-0.29 < 0,"Imagine a self-contained, hypothetical world w...",") Causal graph: X -> Y, V2 -> Y \n",natural indirect effect,yes,ChatCompletionMessage(content='Step 1) Causal ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8527,8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",...,Y_{X=0} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 0 or 1,1,"Imagine a self-contained, hypothetical world w...",") Causal graph: \nX -> Y,\nV2 -> 

In [5]:
def get_graph(text):
    for i in range(len(text)):
        if text[i] == ':':
            break

    text = text[i+1:]

    return text.replace(' ','').replace('\n','').replace('.','').replace('\\n','')

In [35]:
b=') Causal graph: X -> Y, Y -> V2.\n\n'
b=get_graph(b)

In [36]:
b.split(',')

['X->Y', 'Y->V2']

In [24]:
x = ['i','j']
y = ['j','i']
set(x) == set(y)

True

In [6]:
acc = 0

for i in range(len(data)):
    if type(data['predicted graph'].iloc[i]) == str:
        #print(i)
        pred = get_graph(data['predicted graph'].iloc[i]).split(',')
        true_graph = data['step1'].iloc[i].split(',')

        #print(pred,true_graph)
        if set(pred) == set(true_graph):
            acc += 1

acc/len(data)

0.10864978902953587

In [6]:
acc = 0
for i in range(len(data)):
    if data['predicted final answer'].iloc[i] == data['answer'].iloc[i]:
        acc +=1

acc/len(data)

0.5951711204875761

In [10]:
#query acc
acc = 0
l = []

for i in range(len(data)):
    if data["step2"].iloc[i] == data["predicted query type"].iloc[i]:
        acc+=1
        if data["step2"].iloc[i] not in l:
            l.append(data["step2"].iloc[i])

acc/len(data)

0.5666901078293484

In [3]:
from sklearn.metrics import f1_score
f1_score(data['answer'], data["predicted final answer"],average="weighted")

0.5941014153925488

In [8]:
data=data.dropna()
lista = list(data["predicted query type"].unique())
#np.append(lista,'normal counterfactual question')
lista.append("normal counterfactual question")

dicio = dict(zip(lista,range(100)))

data["predicted query type"].replace(dicio)
data["step2"].replace(dicio)

/tmp/ipykernel_1058675/1097159102.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["predicted query type"].replace(dicio)
/tmp/ipykernel_1058675/1097159102.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["step2"].replace(dicio)


0        5
1        0
2        2
3        0
4        3
        ..
8527    10
8528    10
8529    10
8530    10
8531    10
Name: step2, Length: 8249, dtype: int64

In [3]:
a='oi'
b='suav'
a+','+b

'oi,suav'

In [9]:
from sklearn.metrics import f1_score
f1_score(data['step2'], data["predicted query type"], average='weighted')

0.5648388985360393